In [42]:
import sys
import pandas as pd
import numpy as np
from nltk.book import *
from nltk.corpus import stopwords
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
import random
from nltk import classify
from nltk import NaiveBayesClassifier
from nltk.tokenize import word_tokenize
import seaborn as sns
import pyLDAvis
import pyLDAvis.gensim
sns.set_context('notebook')
import warnings
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from nltk.corpus import stopwords
warnings.filterwarnings('ignore')

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


/usr/local/lib/python3.7/site-packages/botocore/awsrequest.py:624: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  class HeadersDict(collections.MutableMapping):
/usr/local/lib/python3.7/site-packages/gensim/corpora/dictionary.py:11: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, defaultdict
/usr/local/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [43]:
#### get all csv dataframes
US_df = pd.read_csv('youtube-new/USvideos.csv') #USA
CA_df = pd.read_csv('youtube-new/CAvideos.csv') #CANADA
DE_df = pd.read_csv('youtube-new/DEvideos.csv') #GERMANY
FR_df = pd.read_csv('youtube-new/FRvideos.csv') #FRANCE
GB_df = pd.read_csv('youtube-new/GBvideos.csv') #GREAT BRITAIN
IN_df = pd.read_csv('youtube-new/INvideos.csv') #INDIA

# JP_df = pd.read_csv('youtube-new/JPvideos.csv', encoding="UTF-8") #JAPAN

# KR_df = pd.read_csv('youtube-new/KRvideos.csv') #SOUTH KOREA

# MX_df = pd.read_csv('youtube-new/MXvideos.csv') #MEXICO

# RU_df = pd.read_csv('youtube-new/RUvideos.csv') #RUSSIA


full_df = pd.concat([US_df, CA_df, DE_df, FR_df, GB_df, IN_df])




In [56]:
###get most common tags
def get_most_common_tags(country_df):
    tags = country_df['tags'].to_string(index=False, header=False)
    split_tags = [i.replace('"', '') for i in tags.split("|")]
    stop_words = stopwords.words('english')
    filtered_tags = [word for word in split_tags if word not in stop_words]
    fdist = FreqDist(split_tags)
    most_popular_tags = fdist.most_common(1000)
    return fdist, dict(most_popular_tags)

fd1, us_most_common_tags = get_most_common_tags(US_df)
f2, ca_most_common_tags = get_most_common_tags(CA_df)
fd3, de_most_common_tags = get_most_common_tags(DE_df)
fd4, fr_most_common_tags = get_most_common_tags(FR_df)
fd5, gb_most_common_tags = get_most_common_tags(GB_df)
fd6, in_most_common_tags = get_most_common_tags(IN_df)

print (us_most_common_tags)



{'comedy': 305, 'Trailer': 272, 'animals': 262, 'how to': 257, 'funny': 253, 'food': 251, 'news': 247, 'basketball': 247, 'Pop': 240, 'music': 222, 'education': 219, 'science': 217, 'animation': 212, 'live': 209, 'official': 209, 'makeup': 203, 'Stephen Colbert': 194, 'vlog': 192, 'Jimmy Fallon': 191, 'Netflix Original Series': 183, 'Happening Now': 180, 'CNN': 179, 'highlights': 171, 'The': 170, 'Colbert': 167, 'Capitol': 165, 'tomscott': 159, 'Review': 154, 'world wrestling entertainment': 152, 'Seth Meyers': 152, 'plays': 150, 'fwf': 148, 'firstwefeast': 148, 'NBC': 148, 'good mythical morning': 147, 'cat': 145, 'TED Ed': 144, 'Records': 143, 'CH originals': 143, 'vox': 140, 'life noggin youtube': 139, 'espn live': 139, 'Saturday Night Live': 135, 'Hank': 135, 'Green': 135, 'hannah': 133, 'talk': 131, 'dude perfect stereotypes': 131, 'gmm': 130, 'Football': 129, 'screen junkies': 129, 'beauty': 128, 'gbs': 128, 'lag': 128, 'documentary': 128, 'television': 125, 'comics': 125, 'comic

In [11]:
####do visualization for most popular tags
'''
word cloud 
bar graph 
'''

'\nword cloud \nbar graph \n'

In [12]:
## do sentiment analysis on each of the tags
positive_words_df = pd.read_fwf('positivewords.txt')
negative_words_df = pd.read_fwf('negativewords.txt')

def extract_features(words):
    return dict([(word, True) for word in words.split()])

def build_sentiment_analysis_model():
    positive_words = positive_words_df['positivewords'].values.tolist()
    negative_words = negative_words_df['negativewords'].values.tolist()
    pos_feats = [(extract_features(f), 'positive') for f in positive_words ]
    neg_feats = [(extract_features(f), 'negative') for f in negative_words ]
    dataset = pos_feats + neg_feats
    random.shuffle(dataset)
    cutoff = int(0.80 * len(dataset))
    train_data = dataset[:cutoff]
    test_data = dataset[cutoff:]

    classifier = NaiveBayesClassifier.train(train_data)
    # print("Accuracy is:", classify.accuracy(classifier, test_data))
    # print(classifier.show_most_informative_features(10))
    return classifier

def execute_model(tags):
    classifications = {}
    classifier = build_sentiment_analysis_model()
    classifier.show_most_informative_features(5)
    for tag in tags:
        classified = classifier.classify(extract_features(tag))
        classifications[tag] = classified
    return classifications



In [13]:
classifications_us = execute_model(list(us_most_common_tags.keys()))
classifications_ca = execute_model(list(ca_most_common_tags.keys()))
classifications_de = execute_model(list(de_most_common_tags.keys()))
classifications_fr = execute_model(list(fr_most_common_tags.keys()))
classifications_gb = execute_model(list(gb_most_common_tags.keys()))
classifications_in = execute_model(list(in_most_common_tags.keys()))

Most Informative Features
             enviousness = True           positi : negati =      2.4 : 1.0
                 coolest = None           negati : positi =      1.0 : 1.0
               righteous = None           negati : positi =      1.0 : 1.0
                  snazzy = None           negati : positi =      1.0 : 1.0
                 happily = None           negati : positi =      1.0 : 1.0
Most Informative Features
                 coolest = None           negati : positi =      1.0 : 1.0
               righteous = None           negati : positi =      1.0 : 1.0
                  snazzy = None           negati : positi =      1.0 : 1.0
                 happily = None           negati : positi =      1.0 : 1.0
                  firmer = None           negati : positi =      1.0 : 1.0
Most Informative Features
             enviousness = True           positi : negati =      2.4 : 1.0
                 coolest = None           negati : positi =      1.0 : 1.0
               righteo

In [14]:
def get_sentiment_stats(classification, country):
    sentiments = list(classification.values())
    sentiments_df = pd.DataFrame(sentiments, columns=['Sentiment'])
    negatives = len(sentiments_df[sentiments_df['Sentiment'] =='negative'])
    positives = len(sentiments_df[sentiments_df['Sentiment'] =='positive'])
    total_len = len(sentiments_df)
    percentage_of_negative = negatives / total_len * 100
    percentage_of_positive = positives / total_len * 100
    print ("{}% of the tags found in the trending youtube videos for country {} were positive in nature, while {}% were negative in nature.\n".format(percentage_of_positive, country, percentage_of_negative))
    
get_sentiment_stats(classifications_us, "USA")
get_sentiment_stats(classifications_ca, "Canada")
get_sentiment_stats(classifications_de, "Denmark")
get_sentiment_stats(classifications_fr, "France")
get_sentiment_stats(classifications_gb, "Great Britain")
get_sentiment_stats(classifications_in, "India")

2.5% of the tags found in the trending youtube videos for country USA were positive in nature, while 97.5% were negative in nature.

3.1% of the tags found in the trending youtube videos for country Canada were positive in nature, while 96.89999999999999% were negative in nature.

1.0999999999999999% of the tags found in the trending youtube videos for country Denmakr were positive in nature, while 98.9% were negative in nature.

1.3% of the tags found in the trending youtube videos for country France were positive in nature, while 98.7% were negative in nature.

1.3% of the tags found in the trending youtube videos for country Great Britain were positive in nature, while 98.7% were negative in nature.

1.9% of the tags found in the trending youtube videos for country India were positive in nature, while 98.1% were negative in nature.



In [15]:
## do visualization of sentiment analysis

In [30]:
def do_LDA_model(country_tags):
    tokenizer = RegexpTokenizer(r'\w+')

    # create English stop words list
    stop_words = stopwords.words('english')

    # Create p_stemmer of class PorterStemmer
    p_stemmer = PorterStemmer()
    # compile sample documents into a list
    tags = list(country_tags.keys())

    # list for tokenized documents in loop
    texts = []

    # loop through document list
    for i in tags:

        # clean and tokenize document string
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)

        # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in stop_words]

        # stem tokens
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]

        # add tokens to list
        texts.append(stemmed_tokens)

    # turn our tokenized documents into a id <-> term dictionary
    dictionary = corpora.Dictionary(texts)

    # convert tokenized documents into a document-term matrix
    corpus = [dictionary.doc2bow(text) for text in texts]

    # generate LDA model
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=100)
    print(ldamodel.print_topics(num_topics=3, num_words=10))

In [62]:
do_LDA_model(us_most_common_tags)
print("-----------------------------------------------------")
do_LDA_model(ca_most_common_tags)
print("-----------------------------------------------------")
do_LDA_model(de_most_common_tags)
print("-----------------------------------------------------")
do_LDA_model(fr_most_common_tags)
print("-----------------------------------------------------")
do_LDA_model(gb_most_common_tags)
print("-----------------------------------------------------")
do_LDA_model(in_most_common_tags)

### ----visualize maybe using pyLDAavis and maybe in a tabular format in a pandas df to show top topics

[(0, '0.017*"news" + 0.010*"video" + 0.010*"makeup" + 0.009*"music" + 0.007*"scienc" + 0.007*"beauti" + 0.006*"last" + 0.006*"iphon" + 0.006*"tutori" + 0.006*"pop"'), (1, '0.012*"anim" + 0.007*"sport" + 0.007*"film" + 0.007*"entertain" + 0.006*"one" + 0.006*"super" + 0.006*"puth" + 0.006*"diy" + 0.006*"review" + 0.006*"lip"'), (2, '0.017*"show" + 0.012*"movi" + 0.011*"new" + 0.010*"buzzfe" + 0.010*"cat" + 0.009*"live" + 0.009*"game" + 0.008*"late" + 0.008*"youtub" + 0.008*"video"')]
-----------------------------------------------------
[(0, '0.025*"news" + 0.007*"buzzfe" + 0.007*"ab" + 0.006*"cbn" + 0.006*"onlin" + 0.006*"jame" + 0.006*"laugh" + 0.006*"fail" + 0.006*"dr" + 0.006*"footbal"'), (1, '0.017*"tv" + 0.017*"show" + 0.013*"live" + 0.012*"nba" + 0.011*"highlight" + 0.009*"late" + 0.008*"trump" + 0.008*"anim" + 0.007*"world" + 0.007*"react"'), (2, '0.012*"video" + 0.010*"movi" + 0.008*"game" + 0.008*"fox" + 0.008*"2018" + 0.007*"sport" + 0.007*"funni" + 0.007*"youtub" + 0.006*"رم

In [50]:
import requests
def do_search_youtube_request(videoId):
    f = open("apiKey", "r")
    key = f.read()
    url = "https://www.googleapis.com/youtube/v3/search?part=snippet&maxResults=2&relatedToVideoId={}&type=video&key={}".format(videoId, key)
    r = requests.get(url)
    return r.json()

In [67]:
import requests
def find_video_insights(videoIds):
    f = open("apiKey", "r")
    key = f.read()
    url = 'https://www.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id=Ks-_Mh1QhMc%2Cc0KYU2j0TM4%2CeIho2S0ZahI&key={}'.format(key)
    r = requests.get(url)
    return r.json()

In [33]:
## call this with 1 country at a time 
import json
def process_youtube_requests(videoIds):
    relatedVideoIds = []
    for videoId in videoIds:
        response = do_search_youtube_request(videoId)
        relatedVideoId = response['items'][0]['id']['videoId']
        relatedVideoIds += relatedVideoId
    videoIdsStr = '%2C'.join([str(elem) for elem in relatedVideoIds])
    r = find_video_insights(videoIdsStr)
    parse_and_visualize(r)

P_If9xMyl10


In [83]:
def parse_and_visualize(r):
    tags = (r['items'][2]['snippet']['tags'])
    channelTitles = (r['items'][2]['snippet']['channelTitle'])
    descriptions = (r['items'][2]['snippet']['description'])
    views = (r['items'][0]['statistics']['viewCount'])
    likes = (r['items'][0]['statistics']['likeCount'])
    dislikes = (r['items'][0]['statistics']['dislikeCount'])
    favorites = (r['items'][0]['statistics']['favoriteCount'])
    comments = (r['items'][0]['statistics']['commentCount'])

7826
